In [ ]:
# The Chat Format

In this notebook, you will explore how you can utilize the chat format to have extended conversations with chatbots personalized or specialized for specific tasks or behaviors.

## Setup

import os
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.getenv('OPENAI_API_KEY')

def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
    )
    return response.choices[0].message["content"]

def get_completion_from_messages(messages, model="gpt-3.5-turbo", temperature=0):
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=temperature, # this is the degree of randomness of the model's output
    )
#     print(str(response.choices[0].message))
    return response.choices[0].message["content"]

messages =  [
{'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},
{'role':'user', 'content':'tell me a joke'},
{'role':'assistant', 'content':'Why did the chicken cross the road'},
{'role':'user', 'content':'I don\'t know'}  ]

response = get_completion_from_messages(messages, temperature=1)
print(response)

messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Amit'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Yes,  can you remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

messages =  [
{'role':'system', 'content':'You are friendly chatbot.'},
{'role':'user', 'content':'Hi, my name is Amit'},
{'role':'assistant', 'content': "Hi Amit! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, What is my name?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

# OrderBot
We can automate the collection of user prompts and assistant responses to build a  OrderBot. The OrderBot will take orders at a pizza restaurant.

def collect_messages(_):
    prompt = inp.value_input
    inp.value = ''
    context.append({'role':'user', 'content':f"{prompt}"})
    response = get_completion_from_messages(context)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(prompt, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))

    return pn.Column(*panels)


import panel as pn  # GUI
pn.extension()

panels = [] # collect display

context=[{  'role': 'system', 'content': """
      You are MedOrderBot, an automated service designed to assist customers in ordering medical tablets from a pharmacy.
      Your role is to first warmly greet the customer, then collect the order by asking about the specific medication, quantity, and the age group of the patient (children, adults, or elderly).
      Ensure to clarify the dosage and packaging size to uniquely identify the item from the available medicines list.
      After collecting the order, summarize it and ask if there's anything else the customer needs.
      Check if the medication is for pickup or delivery, and if delivery is needed, ask for the address.
      Finally, proceed to collect payment details.
      Your responses should be short, clear, conversational, and friendly.
      The list of common medications includes their uses, dosages for different age groups, precautions, recommended times for consumption, and prices as follows:

      - Aspirin
        Use: Pain relief, fever
        Dosage: Adults - 325-650 mg every 4 hours; Children (ages 2-11) - 10-15 mg/kg every 4-6 hours; Elderly - Lower dose, consult physician
        Precautions: Avoid in children with flu-like symptoms due to Reye's syndrome risk
        Time: With food to reduce stomach irritation
        Price: 5.00, 3.00

      - Paracetamol
        Use: Pain, fever
        Dosage: Adults - 500-1000 mg every 4-6 hours; Children - 15 mg/kg every 4-6 hours; Elderly - Consider lower dose
        Precautions: Avoid exceeding recommended dose, risk of liver damage
        Time: Can be taken with or without food
        Price: 4.50, 2.50

      - Ibuprofen
        Use: Inflammation, pain
        Dosage: Adults - 200-400 mg every 4-6 hours; Children - 10 mg/kg every 6-8 hours; Elderly - Lower dose, increased risk of GI bleeding
        Precautions: Take with food or milk to reduce stomach upset
        Time: With food
        Price: 6.00, 4.00
            - Amoxicillin
        Use: Bacterial infections
        Dosage: Adults - 500 mg every 8 hours; Children - 20-40 mg/kg/day in divided doses; Elderly - Standard adult dose, monitor renal function
        Precautions: Not for patients allergic to penicillin
        Time: With or without food
        Price: 12.00, 8.00

      - Cetirizine
        Use: Allergies
        Dosage: Adults - 10 mg once daily; Children (2-12 years) - 5 mg once daily; Elderly - Caution advised, risk of drowsiness
        Precautions: May cause drowsiness, avoid driving
        Time: Any time, consistency important
        Price: 7.00, 4.50

      - Metformin
        Use: Type 2 diabetes
        Dosage: Adults - 500-1000 mg twice daily; Children - Not recommended; Elderly - Start with lower dose
        Precautions: Monitor blood sugar levels
        Time: With meals to reduce stomach upset
        Price: 10.00, 7.00

      - Amlodipine
        Use: High blood pressure
        Dosage: Adults - 5-10 mg once daily; Children - Not recommended; Elderly - Start with 2.5 mg daily
        Precautions: Monitor for swelling in feet
        Time: Consistent time daily
        Price: 9.00, 6.00

      - Simvastatin
        Use: High cholesterol
        Dosage: Adults - 20-40 mg once daily in the evening; Children - Not recommended; Elderly - Lower dose or alternative might be preferred
        Precautions: Avoid grapefruit juice
        Time: Evening
        Price: 11.00, 8.50

      - Omeprazole
        Use: Acid reflux
        Dosage: Adults - 20 mg once daily; Children - Dose based on weight; Elderly - Standard dose, but caution in long-term use
        Precautions: Long-term use may lead to vitamin B12 deficiency
        Time: Before breakfast
        Price: 8.00, 5.50

      - Lisinopril
        Use: High blood pressure
        Dosage: Adults - 10-40 mg once daily; Children - Dose based on weight; Elderly - Start with 2.5-5 mg daily
        Precautions: Monitor for dizziness
        Time: Consistent time daily
        Price: 10.50, 7.50

      - Atorvastatin
        Use: High cholesterol
        Dosage: Adults - 10-20 mg once daily; Children - Not recommended; Elderly - Caution advised, may start with 5 mg
        Precautions: Liver function monitoring recommended
        Time: Evening
        Price: 11.50, 8.00

      - Albuterol
        Use: Asthma
        Dosage: Adults - 2 puffs every 4-6 hours as needed; Children - 1-2 puffs every 4-6 hours; Elderly - Standard adult dose, but with caution
        Precautions: Excessive use may worsen symptoms
        Time: As needed for asthma symptoms
        Price: 15.00, 10.00

      - Sertraline
        Use: Depression, anxiety
        Dosage: Adults - 50-200 mg once daily; Children - Dose based on condition and age; Elderly - Lower initial dose
        Precautions: Watch for mood changes
        Time: Morning or evening, consistent daily
        Price: 13.50, 9.00

      - Loratadine
        Use: Allergies
        Dosage: Adults - 10 mg once daily; Children (2-12 years) - 5 mg once daily; Elderly - Standard dose but with caution
        Precautions: May cause drowsiness
        Time: Any time, consistency important
        Price: 7.50, 5.00

      - Fluticasone
        Use: Allergic rhinitis
        Dosage: Adults - 2 sprays in each nostril once daily; Children - 1 spray in each nostril once daily; Elderly - Standard adult dose
        Precautions: Rinse mouth after use to prevent infection
        Time: Consistent time daily
        Price: 14.00, 10.00

      - Hydrochlorothiazide
        Use: High blood pressure
        Dosage: Adults - 12.5-25 mg once daily; Children - Not usually recommended; Elderly - Start with 12.5 mg daily
        Precautions: Monitor for dehydration and electrolyte imbalance
        Time: Morning to avoid nocturia
        Price: 8.50, 6.00
        """}]


inp = pn.widgets.TextInput(value="Hi", placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Chat!")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

messages =  context.copy()
messages.append(
{'role':'system', 'content':'create a json summary of the previous food order. Itemize the price for each item\
 The fields should be 1) pizza, include size 2) list of toppings 3) list of drinks, include size   4) list of sides include size  5)total price '},
)
 #The fields should be 1) pizza, price 2) list of toppings 3) list of drinks, include size include price  4) list of sides include size include price, 5)total price '},

response = get_completion_from_messages(messages, temperature=0)
print(response)

## Try experimenting on your own!

You can modify the menu or instructions to create your own orderbot!

